# PyFunc Model + Transformer Example

This notebook demonstrates how to deploy a Python function based model and a custom transformer. This type of model is useful as user would be able to define their own logic inside the model as long as it satisfy contract given in `merlin.PyFuncModel`. If the pre/post-processing steps could be implemented in Python, it's encouraged to write them in the PyFunc model code instead of separating them into another transformer.accepts a tensor input. The transformer has preprocessing step that allows the user to send a raw image data and convert it to a tensor input.accepts a tensor input. The transformer has preprocessing step that allows the user to send a raw image data and convert it to a tensor input.

The model we are going to develop and deploy is a cifar10 model accepts a tensor input. The transformer has preprocessing step that allows the user to send a raw image data and convert it to a tensor input.

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

In [1]:
!pip install --upgrade -r requirements.txt > /dev/null

You are using pip version 19.0.3, however version 21.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import warnings
warnings.filterwarnings('ignore')

## 1. Initialize Merlin

### 1.1 Set Merlin Server

In [3]:
import merlin

MERLIN_URL = "<MERLIN_HOST>/api/merlin"

merlin.set_url(MERLIN_URL)

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project-name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP UI.

In [4]:
PROJECT_NAME = "sample"

merlin.set_project(PROJECT_NAME)

/Users/ariefrahmansyah/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in Merlin is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by Merlin are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [5]:
from merlin.model import ModelType

MODEL_NAME = "transformer-pyfunc"

merlin.set_model(MODEL_NAME, ModelType.PYFUNC)

## 2. Train Model

In this step, we are going to train a cifar10 model using PyToch and create PyFunc model class that does the prediction using trained PyTorch model.

### 2.1 Prepare Training Data

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

/Users/ariefrahmansyah/.pyenv/versions/3.7.3/lib/python3.7/site-packages/torchvision/datasets/lsun.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
0it [00:00, ?it/s]

100%|█████████▉| 170434560/170498071 [02:59<00:00, 1240089.84it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


### 2.2 Create PyTorch Model

In [7]:
import torch.nn as nn
import torch.nn.functional as F

class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

/Users/ariefrahmansyah/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 2.3 Train Model

In [8]:
import torch.optim as optim

net = PyTorchModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

170500096it [03:10, 1240089.84it/s]                               

[1,  2000] loss: 2.176
[1,  4000] loss: 1.892
[1,  6000] loss: 1.704
[1,  8000] loss: 1.605
[1, 10000] loss: 1.533
[1, 12000] loss: 1.485
[2,  2000] loss: 1.421
[2,  4000] loss: 1.395
[2,  6000] loss: 1.356
[2,  8000] loss: 1.324
[2, 10000] loss: 1.311
[2, 12000] loss: 1.278


### 2.4 Check Prediction

In [9]:
dataiter = iter(trainloader)
inputs, labels = dataiter.next()

predict_out = net(inputs[0:1])
predict_out

tensor([[-0.4970,  0.6247, -0.7129, -0.4137, -1.6248, -1.0954, -1.6954,  0.9694,
         -1.0732,  3.7799]], grad_fn=<AddmmBackward>)

### 2.5 Serialize Model

In [10]:
import os

model_dir = "pytorch-model"
model_path = os.path.join(model_dir, "model.pt")
model_class_path = os.path.join(model_dir, "model.py")

torch.save(net.state_dict(), model_path)

### 2.6 Save PyTorchModel Class

We also need to save the PyTorchModel class and upload it to Merlin alongside the serialized trained model. The next cell will write the PyTorchModel we defined above to `pytorch-model/model.py` file.

In [11]:
%%file pytorch-model/model.py
import torch.nn as nn
import torch.nn.functional as F

class PyTorchModel(nn.Module):
    def __init__(self):
        super(PyTorchModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Overwriting pytorch-model/model.py


## 3. Create PyFunc Model

To create a PyFunc model you'll have to extend `merlin.PyFuncModel` class and implement its `initialize` and `infer` method.

`initialize` will be called once during model initialization. The argument to `initialize` is a dictionary containing a key value pair of artifact name and its URL. The artifact's keys are the same value as received by `log_pyfunc_model`.

`infer` method is the prediction method that is need to be implemented. It accept a dictionary type argument which represent incoming request body. `infer` should return a dictionary object which correspond to response body of prediction result.

In following example we are creating PyFunc model called `CifarModel`. In its `initialize` method we expect 2 artifacts called `model_path` and `model_class_path`, those 2 artifacts would point to the serialized model and the PyTorch model class file. The `infer` method will simply does prediction for the model and return the result.

In [12]:
import importlib
import sys

from merlin.model import PyFuncModel

MODEL_CLASS_NAME="PyTorchModel"

class CifarModel(PyFuncModel):
    def initialize(self, artifacts):
        model_path = artifacts["model_path"]
        model_class_path = artifacts["model_class_path"]
        
        # Load the python class into memory
        sys.path.append(os.path.dirname(model_class_path))
        modulename = os.path.basename(model_class_path).split('.')[0].replace('-', '_')
        model_class = getattr(importlib.import_module(modulename), MODEL_CLASS_NAME)
        
        # Make sure the model weight is transform with the right device in this machine
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
        self._pytorch = model_class().to(device)
        self._pytorch.load_state_dict(torch.load(model_path, map_location=device))
        self._pytorch.eval()
        
    def infer(self, request, **kwargs):
        inputs = torch.tensor(request["instances"])
        result = self._pytorch(inputs)
        return {"predictions":  result.tolist()}

Now, let's test it locally.

In [13]:
import json

with open(os.path.join("input-tensor.json"), "r") as f:
    tensor_req = json.load(f)

m = CifarModel()
m.initialize({"model_path": model_path, "model_class_path": model_class_path})
m.infer(tensor_req)

{'predictions': [[0.555002748966217,
   -1.7436292171478271,
   1.391134262084961,
   1.4474482536315918,
   -0.8920332193374634,
   0.520797610282898,
   0.13903649151325226,
   -1.9986869096755981,
   1.243778109550476,
   -1.5127893686294556]]}

## 4. Deploy Model

To deploy the model, we will have to create an iteration of the model (by create a `model_version`), upload the serialized model to MLP, and then deploy.

### 4.1 Create Model Version and Upload

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_pyfunc_model(model_instance=EnsembleModel(), 
                conda_env="env.yaml", 
                artifacts={"xgb_model": model_1_path, "sklearn_model": model_2_path})
v.finish()
```

To upload PyFunc model you have to provide following arguments:
1. `model_instance` is the instance of PyFunc model, the model has to extend `merlin.PyFuncModel`
2. `conda_env` is path to conda environment yaml file. The environment yaml file must contain all dependency required by the PyFunc model.
3. (Optional) `artifacts` is additional artifact that you want to include in the model
4. (Optional) `code_path` is a list of directory containing python code that will be loaded during model initialization, this is required when `model_instance` depend on local python package

In [14]:
with merlin.new_model_version() as v:    
    merlin.log_pyfunc_model(model_instance=CifarModel(),
                            conda_env="env.yaml",
                            artifacts={"model_path": model_path, "model_class_path": model_class_path})

2021/06/23 05:41:28 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under gs://<MERLIN_BUCKET>/mlflow/604/7b57180c051842fe815adbacfa282541/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the  tracking store. If logging to a mlflow server via REST, consider  upgrading the server version to MLflow 1.7.0 or above.


### 4.2 Deploy Model and Transformer

To deploy a model and its transformer, you must pass a `transformer` object to `deploy()` function. Each of deployed model version will have its own generated url.

In [15]:
from merlin.resource_request import ResourceRequest
from merlin.transformer import Transformer

# Create a transformer object and its resources requests
resource_request = ResourceRequest(min_replica=1, max_replica=1, 
                                   cpu_request="100m", memory_request="200Mi")
transformer = Transformer("gcr.io/kubeflow-ci/kfserving/image-transformer:latest",
                          resource_request=resource_request)

endpoint = merlin.deploy(v, transformer=transformer)

/Users/ariefrahmansyah/.pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Deploying model transformer-pyfunc version 2
0% [##############################] 100% | ETA: 00:00:00

Model transformer-pyfunc version 2 is deployed.
View model version logs: http://<MERLIN_HOST>/merlin/projects/1/models/601/versions/2/endpoints/f3c84055-1c4f-4b6c-aa7c-9c494024cc3f/logs



Total time elapsed: 00:07:51


### 4.3 Send Test Request

In [16]:
import json
import requests

with open(os.path.join("input-raw-image.json"), "r") as f:
    req = json.load(f)

resp = requests.post(endpoint.url, json=req)
resp.text

'{"predictions": [[0.555002748966217, -1.7436292171478271, 1.391134262084961, 1.4474482536315918, -0.8920332193374634, 0.520797610282898, 0.13903649151325226, -1.9986869096755981, 1.243778109550476, -1.5127893686294556]]}'

## 4. Clean Up

## 4.1 Delete Deployment

In [17]:
merlin.undeploy(v)

Deleting deployment of model transformer-pyfunc version 2 from enviroment id-staging
